In [11]:
import random

import nltk
from pymorphy2 import MorphAnalyzer
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.decomposition import TruncatedSVD
import pandas as pd
from sklearn.metrics.pairwise import cosine_similarity

from globals import create_tables, session_factory
from models.Question import Question
from src.normalizer import normalize_str

In [12]:
create_tables()

In [13]:
nltk.download("punkt")
nltk.download("stopwords")

[nltk_data] Error loading punkt: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>
[nltk_data] Error loading stopwords: <urlopen error [SSL:
[nltk_data]     CERTIFICATE_VERIFY_FAILED] certificate verify failed:
[nltk_data]     unable to get local issuer certificate (_ssl.c:997)>


False

In [14]:
# QUESTION_TO_SEARCH = "Часто я бросаю даже важную деятельность, если усилия, которые требуются, становятся слишком неприятными."
QUESTION_TO_SEARCH = "Вы считаете себя недооцененным?"
normalized_question = normalize_str(QUESTION_TO_SEARCH, MorphAnalyzer())

In [15]:
with session_factory() as session:
    question_pair = [(QUESTION_TO_SEARCH, normalized_question)] + session.query(Question.raw, Question.transformed).all()
df = pd.DataFrame(question_pair, columns=['raw', 'transformed'])
df.head(10)

,raw,transformed
0,Вы считаете себя недооцененным?,считать недооценить
1,У Вас есть тяга к рисованию?,тяга рисование
2,"Я все время ищу способ, который поможет мне за...",время искать способ который помочь забыться пе...
3,Я чувствую себя сексуально непривлекательным,чувствовать сексуально непривлекательный
4,"Я чувствую, как жизнь уходит из меня по капле",чувствовать жизнь уходить капля
5,Я чувствую себя лишним человеком в своей семье,чувствовать лишний человек свой семья
6,Единственная возможность достойно пройти свой ...,единственный возможность достойно пройти свой ...
7,"Я не доволен тем, как сложилась моя жизнь",довольный сложиться жизнь
8,"Потери невосполнимы, и если уйдут из моей жизн...",потеря невосполнимый уйти мой жизнь близкий че...
9,"Семья — это рабство, одиночество — это свобода",семья это рабство одиночество это свобода


# Helpers

In [16]:
def print_closest(model_df: pd.DataFrame):
    cos_sim = cosine_similarity(model_df)
    enum_cos_sim = enumerate(cos_sim[0])
    enum_sort_cos_sim = sorted(enum_cos_sim, key=lambda x: x[1], reverse=True)
    print(QUESTION_TO_SEARCH, end="\n\n")
    for i, score in enum_sort_cos_sim[1: 10]:
        print(score, df["raw"][i], sep=": ")

# Tf-Idf

In [17]:
tfidf_vect = TfidfVectorizer(
    analyzer='word',
    max_df=0.95,
    min_df=3,
    ngram_range=(1, 2),
    max_features=100_000,
    stop_words=nltk.corpus.stopwords.words('russian')
)
tfidf_model = pd.DataFrame.sparse.from_spmatrix(
    tfidf_vect.fit_transform(df['transformed'])
)

In [18]:
print_closest(tfidf_model)

Вы считаете себя недооцененным?

1.0: Я считаю, что не грех иногда пожалеть самого себя.
0.6947116602249878: Вас считают хорошим рассказчиком?
0.6496336888070184: Я считаю, что при неудачах нельзя отчаиваться.
0.550434878470806: Считаю, что я ничем не отличаюсь от большинства людей.
0.5380255967589899: Я считаю, что жизнь справедлива ко мне
0.49382294508472385: Я считаю себя виноватым перед родителями.
0.4908996451792499: Другие находят у меня крупные недостатки, я считаю, что они преувеличивают.
0.48454859720231064: Я считаю, что никто не должен отрываться от коллектива.
0.4834283477724685: Я не считаю, что достаточно духовно интересен для того, чтобы быть притягательным для многих людей.


# LSA

In [19]:
# TODO: CountVectoriser
lsa = TruncatedSVD(
    n_components=100,
    n_iter=1000
)
lsa_model = lsa.fit_transform(tfidf_model)

In [20]:
print_closest(lsa_model)

Вы считаете себя недооцененным?

1.0000000000000007: Я считаю, что не грех иногда пожалеть самого себя.
0.7915766379057013: Считаю, что я ничем не отличаюсь от большинства людей.
0.7507032865039938: Я считаю, что жизнь справедлива ко мне
0.7424893041569376: Где-то в глубине души я считаю себя слабаком.
0.7175775324114347: Я считаю, что при неудачах нельзя отчаиваться.
0.7065661481974279: Другие находят у меня крупные недостатки, я считаю, что они преувеличивают.
0.7049635056948035: Вас считают хорошим рассказчиком?
0.6538421550356885: Я считаю себя виноватым перед родителями.
0.6016834337365947: Я считаю, что никто не должен отрываться от коллектива.
